In [3]:
from superai.nn.layer.fc import FullyConnected
from superai.nn.layer.activator import Activator
from superai.nn.model.nnet import Sequence
import numpy as np
import os
import struct
from superai.nn.optimizer.optimizer import Adam

In [ ]:
def convert_to_one_hot(y, C):
    return np.eye(C)[y.reshape(-1)].T

def calculate_accuracy(model, X, Y):
    m, n = X.shape
    Y_pre = model.predict(X)
    Y_pre = Y_pre == np.max(Y_pre, axis=0, keepdims = True)
    C = np.abs(Y_pre - Y)
    error = np.sum(C) / 2
    print(1 - error / n)

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)
    return images, labels


X_train, Y_train = load_mnist(path="MNIST")
X_train = X_train.transpose()
Y_train = convert_to_one_hot(Y_train, 10)
m, n = X_train.shape
# normalization fc-layer inputs
nn_input_min = np.min(X_train, axis=1, keepdims=True)
nn_input_max = np.max(X_train, axis=1, keepdims=True)
nn_input_final = ((X_train - nn_input_min) / (nn_input_max - nn_input_min + 1e-11))
X_train = nn_input_final

linearLayer1 = FullyConnected(m, 40)
linearLayer1.layer = 1
activator1 = Activator('tanh')
activator1.layer = 2
linearLayer2 = FullyConnected(40, 20)
linearLayer2.layer = 3
activator2 = Activator('tanh')
activator2.layer = 4
linearLayer3 = FullyConnected(20, 10)
linearLayer3.layer = 5
# outputLayer = NNActivator(linearLayer3, 'softmax', 10)

model = Sequence([linearLayer1, activator1, linearLayer2, activator2, linearLayer3], learning_rate=0.01, iteration_count=100, lambd=0,
                 use_mini_batch=False, mini_batch_size=32)
#adamOpt = Adam()
#adamOpt.run(model, X_train, Y_train)
model.fit(X_train, Y_train)

X_test, Y_test = load_mnist(path="MNIST", kind="t10k")
X_test = X_test.transpose()
Y_test = convert_to_one_hot(Y_test, 10)

calculate_accuracy(model, X_train, Y_train)
calculate_accuracy(model, X_test, Y_test)

print("over")

iteration0  cost:2.3866864108175503, accuracy:0.11519999999999997
iteration1  cost:2.355303712527843, accuracy:0.12008333333333332
iteration2  cost:2.3253782318512126, accuracy:0.12729999999999997
iteration3  cost:2.297041113617258, accuracy:0.1345833333333334
iteration4  cost:2.2703500039795443, accuracy:0.14358333333333329
iteration5  cost:2.24529226718875, accuracy:0.15318333333333334
iteration6  cost:2.2217971545214015, accuracy:0.16310000000000002
iteration7  cost:2.199752693438642, accuracy:0.17420000000000002
iteration8  cost:2.179023154618026, accuracy:0.18594999999999995
iteration9  cost:2.159464171131359, accuracy:0.19886666666666664
iteration10  cost:2.1409340972389925, accuracy:0.2113166666666667
iteration11  cost:2.1233014244793385, accuracy:0.22394999999999998
iteration12  cost:2.106448809526558, accuracy:0.23724999999999996
iteration13  cost:2.0902745603881967, accuracy:0.24978333333333336
iteration14  cost:2.0746924263823945, accuracy:0.2625833333333333
iteration15  cos